In [1]:
import torch.backends.mps

from decoder import AttnDecoderRNN
from encoder import EncoderRNN
from evaluate import evaluate_losses, inference, evaluate_bleu, evaluate_loss
from dataloader import Lang
from io import open
import os
import numpy as np
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu" if torch.backends.mps.is_available() else "cpu")
print(f'using device: {device}')

using device: cuda


In [2]:
initial_validation_size = 20000
max_length = 20
input_lang = Lang("en")
output_lang = Lang("de")

validation_english = open("data/train.en", encoding='utf-8').readlines()[:initial_validation_size]
validation_german = open("data/train.de", encoding='utf-8').readlines()[:initial_validation_size]
zipped = list(zip(validation_english, validation_german))
validation_english = [english for english, german in zipped
                      if len(input_lang.tokenize_without_truncation(english)) < max_length
                      and len(output_lang.tokenize_without_truncation(german)) < max_length]
validation_german = [german for english, german in zipped
                     if len(input_lang.tokenize_without_truncation(english)) < max_length
                     and len(output_lang.tokenize_without_truncation(german)) < max_length]
# validation_german = [[output_lang.decode(token) for token in output_lang.tokenize(sentence)][1:-1] for sentence in validation_german]
print(f"validation size: {len(validation_english)}")

validation size: 2534


In [3]:
losses_list = []

In [4]:
model_names = [
"1e-3_lr_256_hidden_4_layers_20p_dropout",
"1e-4_lr_320_hidden_4_layers_10p_dropout",
"3e-4_lr_320_hidden_4_layers_20p_dropout",
"3e-5_lr_320_hidden_5_layers_30p_dropout",
"50p_tfr_1e-4_lr_320_hidden_4_layers_10p_dropout",
"80p_tfr_3e-4_lr_320_hidden_5_layers_30p_dropout",
"100p_tfr_1e-4_lr_320_hidden_6_layers_40p_dropout",
"100p_tfr_1e-4_lr_512_hidden_8_layers_50p_dropout",
"100p_tfr_2e-4_lr_400_hidden_8_layers_60p_dropout",
"100p_tfr_5e-5_lr_512_hidden_8_layers_60p_dropout_1e-4_weight_decay"]

In [5]:
lstm_model_names = [
"80p_tfr_3e-4_lr_320_hidden_5_layers_30p_dropout",
"50p_tfr_1e-4_lr_320_hidden_4_layers_10p_dropout",
"100p_tfr_1e-4_lr_320_hidden_6_layers_40p_dropout",
"100p_tfr_1e-4_lr_512_hidden_8_layers_50p_dropout",
"100p_tfr_2e-4_lr_400_hidden_8_layers_60p_dropout",
"100p_tfr_5e-5_lr_512_hidden_8_layers_60p_dropout_1e-4_weight_decay"
]

In [6]:
# losses_list = []
for i in range(1):
    models_dir = "models_lstm"
    model_name = lstm_model_names[i]

    encoder = torch.load(os.path.join(models_dir, model_name, "encoder.pt"), map_location=device)
    attn_decoder = torch.load(os.path.join(models_dir, model_name, "decoder.pt"), map_location=device)
    # prev_loss_history = np.load(os.path.join(plots_dir, model_name + "_full_history.npy")).tolist()
    # prev_plot_history = np.load(os.path.join(plots_dir, model_name + "_plot_history.npy")).tolist()

    print(f"Evaluating model {model_name}")
    loss, losses = evaluate_losses(encoder, attn_decoder, validation_english, validation_german, input_lang, output_lang, max_length, device)
    losses_list.append((loss, losses))
    print(f"Loss: {loss}")

Evaluating model 80p_tfr_3e-4_lr_320_hidden_5_layers_30p_dropout


  0%|          | 0/2534 [00:00<?, ?it/s]C:\Users\Lennart\.conda\envs\py310\lib\site-packages\torch\nn\modules\rnn.py:774: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\RNN.cpp:968.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,
100%|██████████| 2534/2534 [01:22<00:00, 30.88it/s]

Loss: 6.043328820083117


In [7]:
len(losses_list)

1

In [9]:
chencherry = SmoothingFunction()
all_bleu_scores = []

for i in range(1):
    models_dir = "models_lstm"
    model_name = lstm_model_names[i]

    encoder = torch.load(os.path.join(models_dir, model_name, "encoder.pt"), map_location=device)
    decoder = torch.load(os.path.join(models_dir, model_name, "decoder.pt"), map_location=device)
    # prev_loss_history = np.load(os.path.join(plots_dir, model_name + "_full_history.npy")).tolist()
    # prev_plot_history = np.load(os.path.join(plots_dir, model_name + "_plot_history.npy")).tolist()

    print(f"Evaluating model {model_name}")
    bleu_scores = []
    for j in tqdm(range(len(validation_english))):
        prediction, _ = inference(encoder, decoder, validation_english[j], input_lang, output_lang, max_length, device)
        bleu_score = sentence_bleu(validation_german[j], prediction[1:-1], (0.25, 0.25, 0.25, 0.25), smoothing_function=chencherry.method1)
        bleu_scores.append(bleu_score)
    # print(f"Prediction: {prediction[1:-1]}")
    print(f"BLEU score: {np.mean(bleu_scores)}")
    all_bleu_scores.append(bleu_scores)
    # loss, losses = evaluate_losses(encoder, attn_decoder, validation_english, validation_german, input_lang, output_lang, max_length, device)
    # losses_list.append((loss, losses))
    # print(f"Loss: {loss}")

Evaluating model 80p_tfr_3e-4_lr_320_hidden_5_layers_30p_dropout


100%|██████████| 2534/2534 [01:33<00:00, 27.18it/s]

BLEU score: 0.015802574814471573


In [21]:
np.save("evaluation_loss.npy", losses_list[0][1])
np.save("evaluation_bleu.npy", np.array(all_bleu_scores[0]))

In [24]:
losses = np.load("evaluation_loss.npy", allow_pickle=True)
bleu_scores = np.load("evaluation_bleu.npy", allow_pickle=True)

In [25]:
print(losses.shape)
print(bleu_scores.shape)

(2534,)
(2534,)


In [ ]:
models_of_interest = [4, 5, 8, 9]

In [ ]:
plt.boxplot([losses_list[i][1] for i in models_of_interest])
plt.ylabel("Validation loss")
plt.title("Validation loss for GRU models")
plt.ylim(0, 12)
plt.savefig("plots/losses_boxplot.png", bbox_inches='tight', dpi=300)

In [ ]:
plt.boxplot([all_bleu_scores[i] for i in models_of_interest])
plt.ylabel("BLEU score")
plt.title("BLEU score for GRU models")
# set axis limits
plt.ylim(0.0, 0.12)
plt.savefig("plots/bleu_boxplot.png", bbox_inches='tight', dpi=300)